In [1]:
!pip install fuzzywuzzy

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np
import string

c:\users\varun jadia\appdata\local\programs\python\python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#set up df and target sentence for string matching
triggers = pd.read_csv('weather app sample conversations - more explicit triggers.csv', names = ['Sentences'])
triggers.dropna(inplace=True)
target_sent = 'What is the weather'

In [4]:
#classify based on fuzzy string ratio
triggers['fuzzydist'] = triggers['Sentences'].apply(lambda x: fuzz.ratio(x, target_sent))
triggers['label'] = triggers['fuzzydist'].apply(lambda x: x > 50)

In [26]:
triggers

,Sentences,fuzzydist,label,sent_cleaned,label_cleaned,label_mult
0,What's the weather there?,77,True,Whats the weather there,True,True
1,I wonder what the weather there is,57,True,I wonder what the weather there is,True,True
2,What's the weather going to be?,68,True,Whats the weather going to be,True,True
3,What's the weather going to be this weekend?,54,True,Whats the weather going to be this weekend,True,True
4,What will the weather be?,82,True,What will the weather be,True,True
5,What will be the weather be?,77,True,What will be the weather be,True,True
6,"Weather forecast, please.",32,False,Weather forecast please,False,False
7,"Weather report, please.",33,False,Weather report please,False,False
8,I want to know the weather.,65,True,I want to know the weather,True,True
9,Please tell me whether it'll rain.,45,False,Please tell me whether itll rain,False,False


In [5]:
sum(triggers['label'])/len(triggers)

0.4594594594594595

In [6]:
#clean sentences first and pass into fuzzywuzzy
triggers['sent_cleaned'] = triggers['Sentences'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
triggers['label_cleaned'] = triggers['sent_cleaned'].apply(lambda x: fuzz.ratio(x, target_sent) > 50)

In [7]:
sum(triggers['label_cleaned'])/len(triggers)

0.4594594594594595

In [24]:
#test using multiple trigger sents
target_sents = ['what is the weather', 'will it rain today', 'is it hot today', 'what is todays forecast', 'is it cold outside']
def classify_sent(sent): 
    scores = [fuzz.ratio(sent, target) for target in target_sents]
    print(scores)
    return max(scores) > 50

triggers['label_mult'] = triggers['sent_cleaned'].apply(lambda x: classify_sent(x))
sum(triggers['label_mult'])/len(triggers)

[76, 20, 37, 39, 15]
[60, 23, 24, 35, 15]
[67, 26, 36, 35, 13]
[52, 23, 32, 31, 13]
[79, 43, 26, 43, 19]
[74, 36, 24, 40, 18]
[29, 20, 21, 48, 24]
[30, 26, 22, 32, 26]
[71, 32, 29, 29, 23]
[47, 40, 26, 15, 28]
[50, 17, 32, 38, 51]
[32, 32, 35, 29, 70]
[39, 65, 38, 31, 35]
[36, 14, 15, 34, 25]
[50, 41, 44, 41, 72]
[44, 40, 49, 40, 65]
[56, 21, 40, 28, 37]
[41, 37, 40, 37, 37]
[60, 29, 43, 48, 37]
[29, 48, 31, 31, 19]
[35, 36, 38, 30, 18]
[21, 11, 23, 23, 54]
[29, 54, 31, 34, 21]
[53, 38, 41, 25, 35]
[65, 36, 33, 52, 58]
[24, 50, 43, 36, 65]
[44, 40, 49, 40, 65]
[31, 68, 80, 37, 32]
[29, 43, 32, 38, 26]
[71, 27, 34, 37, 59]
[26, 67, 52, 40, 27]
[29, 39, 42, 74, 24]
[58, 32, 34, 24, 14]
[78, 20, 32, 40, 15]
[29, 37, 31, 27, 48]
[62, 60, 36, 42, 17]
[46, 18, 34, 33, 43]


0.7297297297297297

In [18]:
#test on negative examples
f = open('samples.txt', 'r')

In [19]:
labels_cont = [classify_sent(sent) for sent in f]    

In [22]:
sum(labels_cont)

25

In [25]:
classify_sent("Im headed to New York this summer since my internship is remote anyway. Oh, that sounds cool! I've heard NY gets really humid at that time of the year though? I don't mind a little humidity, I think it'll be fine! Sweet! Let me know how that goes - I might have to go to Florida for mine, so well be on the same time zone. Oh, awesome")

[9, 4, 4, 10, 9]


False